In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
d_tr = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/Titanic/train.csv', index_col='PassengerId')
d_te = pd.read_csv('/Users/antirrabia/Documents/01-GitHub/DataMining-_-/CSV/Titanic/test.csv', index_col=0)

In [3]:
d_tr.sample(7)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
573,1,1,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0,0,PC 17474,26.3875,E25,S
69,1,3,"Andersson, Miss. Erna Alexandra",female,17.0,4,2,3101281,7.9250,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
612,0,3,"Jardin, Mr. Jose Neto",male,NaN,0,0,SOTON/O.Q. 3101305,7.0500,NaN,S
641,0,3,"Jensen, Mr. Hans Peder",male,20.0,0,0,350050,7.8542,NaN,S
565,0,3,"Meanwell, Miss. (Marion Ogden)",female,NaN,0,0,SOTON/O.Q. 392087,8.0500,NaN,S
829,1,3,"McCormack, Mr. Thomas Joseph",male,NaN,0,0,367228,7.7500,NaN,Q


In [4]:
d_te.sample(7)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1083,1,"Salomon, Mr. Abraham L",male,NaN,0,0,111163,26.0000,NaN,S
1018,3,"Brobeck, Mr. Karl Rudolf",male,22.0,0,0,350045,7.7958,NaN,S
925,3,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
1264,1,"Ismay, Mr. Joseph Bruce",male,49.0,0,0,112058,0.0000,B52 B54 B56,S
974,1,"Case, Mr. Howard Brown",male,49.0,0,0,19924,26.0000,NaN,S
1270,1,"Hipkins, Mr. William Edward",male,55.0,0,0,680,50.0000,C39,S
1100,1,"Rosenbaum, Miss. Edith Louise",female,33.0,0,0,PC 17613,27.7208,A11,C


## Tools

In [ ]:
def Fare(s):
    #s['FareBand'] = pd.qcut(s['Fare'], 4)
    s.loc[ s['Fare'] <= 7.91, 'Fare'] = 0
    s.loc[(s['Fare'] > 7.91) & (s['Fare'] <= 14.454), 'Fare'] = 1
    s.loc[(s['Fare'] > 14.454) & (s['Fare'] <= 31), 'Fare']   = 2
    s.loc[ s['Fare'] > 31, 'Fare'] = 3

    #Cambiar tipo de columna
    s['Fare'] = s['Fare'].astype(int)
    
    return s

def Age(s): #if it uses 2 times it will set all values to 1.0
            #recive a copy
            #does not touch nan values
    s.loc[(s.Age <= 13), 'Age'] = 1
    s.loc[(s.Age > 13) & (s.Age <= 18), 'Age'] = 2
    s.loc[(s.Age > 18) & (s.Age <= 30), 'Age'] = 3
    s.loc[(s.Age > 30) & (s.Age <= 50), 'Age'] = 4
    s.loc[(s.Age > 50), 'Age'] = 5
    
    return s

def FamilySize(s):
    s['FamilySize'] = s['SibSp'] +  s['Parch'] + 1
    
    s['FamilySize'] = s['FamilySize'].astype(int)
    #s['Survived'] = s['Survived'].astype(int)
    
    s['IsAlone'] = 0
    s.loc[s['FamilySize'] == 1, 'IsAlone'] = 1
    
    return s.drop(columns=['SibSp', 'Parch', 'FamilySize']).copy()

def Titles(s):
    #str.extract(' ([A-Za-z]+)\.')
    s['Title'] = s['Name'].str.extract(' ([A-Za-z]+)\.')
    s['Title'] = s['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr',                                       
                                    'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
    s['Title'] = s['Title'].replace(['Mlle', 'Ms'], 'Miss')
    s['Title'] = s['Title'].replace(['Mme'], 'Mrs')
    
    return s.drop(columns='Name').copy()

def SexEmbarkedTitle_to_OneHot(s):
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    #s.columns
    
    # ['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    
    cat_features = ['Sex', 'Title']
    res_features = ['Survived', 'Pclass', 'Age', 'Fare', 'Embarked', 'IsAlone']
    
    transformer = ColumnTransformer([('cat', encoder, cat_features)], remainder='passthrough')
    
    transformer.fit(s)
    
    tranformed_names = transformer.named_transformers_['cat'].get_feature_names(cat_features)
    #tranformed_names
    
    new_names = np.append(tranformed_names, res_features)
    
    return pd.DataFrame(transformer.transform(s), columns=new_names)

#temp01 = temp01.drop(columns=['Ticket', 'Cabin'])

def Clean_data(df):
    d_clean = Titles(df)
    d_clean = Fare(d_clean)
    d_clean = Age(d_clean)
    d_clean = FamilySize(d_clean)

    #OneHot aplied
    d_clean.drop(columns=['Ticket', 'Cabin'], inplace=True)

    d_clean = SexEmbarkedTitle_to_OneHot(d_clean)
    
    d_clean['Embarked'] = d_clean['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2})#.astype(int)
        
    return d_clean

# Para prepara el df para age y embarked prediction

def Age_Embarked(df):

    age_X_te = df.loc[df.Age.isna()].copy()
    embarked_X_te = df.loc[df.Embarked.isna()].copy()

    embarked_X_te.drop(columns=['Embarked'], inplace=True)
    age_X_te.drop(columns=['Age'], inplace=True)


    age_X_tr = df.dropna(subset=['Age', 'Embarked']).copy()
    age_y_tr = age_X_tr.Age


    embarked_X_tr = df.dropna(subset=['Age', 'Embarked']).copy()
    embarked_y_tr = embarked_X_tr.Embarked

    # to leave the df with all columns
    # age_X_tr.drop(columns=['Age'], inplace=True)
    # embarked_X_tr.drop(columns=['Embarked'], inplace=True)
    
    return (age_X_te, embarked_X_te, age_X_tr, age_y_tr, embarked_X_tr, embarked_y_tr)

def get_x_y(df, y):
    #resives an copy of a df and return X_tr, y_tr
    X_tr = df.drop(columns=[y])
    y_tr = df[y]
    
    return X_tr, y_tr
